In [1]:
import nltk
import numpy as np
import pandas as pd
import os
import glob
import re as regex
import scipy
from collections import Counter
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize,RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from nltk import wordpunct_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import WordNetLemmatizer 
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
def getFilePath():
  files = []
  path = '/content/drive/MyDrive/Assignment-1-IR-F'
  for d in glob.glob(os.path.join(path, '*')):
    files.append(d)
  return files

In [4]:
def tokenize(text):
  tokens = nltk.word_tokenize(text)
  return tokens

In [5]:
def lowerCase(text):
  text = text.lower()
  return text

In [6]:
def lemmatization(text):
  lemma = lemmatizer.lemmatize(text)
  return lemma

In [7]:
def stopWord(text):
  sentence = []
  stop_words = set(stopwords.words("english"))
  for w in range(0, len(text)):
        if text[w] not in stop_words:
            sentence.append(text[w])
        else:
          continue
  return sentence

In [8]:
def positionalIndex():
  terms = {}
  files = getFilePath()

  for i in range(len(files)):
     # File Path 
      path = files[i]
        
    # Word Position
      wordPos = 0

    # Document ID
      docId = files[i]

    # Open the Current File

      text = open(path,'r',errors='ignore').read()
        
      tokens = nltk.word_tokenize(text)
      tokens = [item for item in tokens if item.isalpha()]
      tokens = stopWord(tokens)

      for item in tokens:
        item = lowerCase(item)
        lemma = lemmatization(item)

        termKeys = terms.keys()             

        if lemma in termKeys :

          #Find the key of the token after lemmatization
          newDict = terms[lemma]
          newDictKeys = terms[lemma].keys()

          #check if docId is present then append the word pos        
          if docId in newDictKeys:
            newList = newDict[docId]
            newList.append(wordPos)
            terms[lemma][docId] = newList

          # if not present then make a new lsit
          # and append the wordPos in list and then in dictionary
          # corresponding to its doc name         
          else:
            newList = []
            newList.append(wordPos)
            terms[lemma][docId] = newList
                    
        # If not present then make a new dict and 
        # append a word pos corresponding to its key            
        else:
          makeNewList = []
          makeNewDict = {}
          makeNewList.append(wordPos)
          makeNewDict[docId] = makeNewList
          terms[lemma] = makeNewDict
            
        wordPos = wordPos + 1
  return terms

In [11]:
def executeQuery(inputquery):
  terms = positionalIndex()
  lemma = []
  finalDict = {}
  count = 0
  # Preprocessing
  tokens = nltk.word_tokenize(inputquery)
  tokens = [item for item in tokens if item.isalpha()]
  tokens = stopWord(tokens)
  for item in tokens:
    print(item)
    item = lowerCase(item)
    lemma += [lemmatization(item)]
  for item in lemma :
    print(item)
    newDict = terms[item]
    if(len(finalDict) > 0):
        tempDict = {}
        documentList = list(set(finalDict.keys()) & set(newDict.keys())) 
        if len(documentList) == 0:
            print("No Documet Found")
            break
        # store doc in a dict and finally store in a final dict
        for document in documentList :
            tempDict[document] = finalDict[document]     
        finalDict = tempDict
        # Again Iterate and find the position 
        for doc in documentList:
            
            list_posting = finalDict[doc]
            temp_pos = []
            
            for pos in list_posting:
                
                if pos+count in newDict[doc]:
                    temp_pos.append(pos)
            
            finalDict[doc] = temp_pos
            
            if len(finalDict[doc]) == 0 :
                del finalDict[doc]
    else:
        finalDict = newDict
    count += 1  
  return finalDict

In [13]:
query = str(input('Enter Query:'))
finalResult = executeQuery(query)
print("Number of Documents Retrived are as Follows:" ,len(finalResult))
finalResult

Enter Query:lion 
lion
lion
Number of Documents Retrived are as Follows: 27


{'/content/drive/MyDrive/Assignment-1-IR-F/bitnet.txt': [1079],
 '/content/drive/MyDrive/Assignment-1-IR-F/boneles2.txt': [121,
  129,
  142,
  792,
  1255,
  1258,
  1330,
  1337,
  1344,
  1374,
  1405,
  1428,
  1583],
 '/content/drive/MyDrive/Assignment-1-IR-F/booze1.fun': [693],
 '/content/drive/MyDrive/Assignment-1-IR-F/christop.int': [1021],
 '/content/drive/MyDrive/Assignment-1-IR-F/collected_quotes.txt': [3650],
 '/content/drive/MyDrive/Assignment-1-IR-F/computer.txt': [204],
 '/content/drive/MyDrive/Assignment-1-IR-F/deep.txt': [593],
 '/content/drive/MyDrive/Assignment-1-IR-F/dromes.txt': [491],
 '/content/drive/MyDrive/Assignment-1-IR-F/drunk.txt': [73],
 '/content/drive/MyDrive/Assignment-1-IR-F/dthought.txt': [608],
 '/content/drive/MyDrive/Assignment-1-IR-F/filmgoof.txt': [5654],
 '/content/drive/MyDrive/Assignment-1-IR-F/hecomes.jok': [245],
 '/content/drive/MyDrive/Assignment-1-IR-F/japantv.txt': [360, 444],
 '/content/drive/MyDrive/Assignment-1-IR-F/lion.jok': [18,
  